# 使用您自己的聲音報告PowerPoint內容的簡報影片生成器

這個 Google Colab Notebook 將引導您完成以下步驟：
1. **安裝必要的套件**：包括用於處理PowerPoint、音訊和影片的程式庫。
2. **上傳檔案**：您需要上傳您的 PowerPoint (.pptx) 檔案和一段您聲音的音訊樣本 (.wav)。
3. **設定 F5-TTS 模型**：這是一個先進的文字轉語音模型，可以用您的聲音樣本來複製語音。
4. **從 PowerPoint 提取筆記**：程式將讀取您在每張投影片筆記區塊中的文字。
5. **生成語音**：使用 F5-TTS 和您上傳的聲音，為每張投影片的筆記生成對應的語音檔。
6. **將 PowerPoint 轉換為圖片**：每張投影片將被存成一張圖片。
7. **合成影片**：將每張投影片的圖片和對應的語音合成為一個影片片段。
8. **合併並下載**：將所有影片片段合併成一個最終的簡報影片，並提供下載連結。

In [ ]:
# @title 1. 安裝所有必要的套件
# @markdown 請執行此儲存格來安裝所有需要的程式庫。這可能需要幾分鐘的時間。
print("正在安裝 F5-TTS 及其相關套件...")
!git clone https://github.com/SWivid/F5-TTS.git
%cd F5-TTS
!pip install -e .
!pip install gradio

print("正在安裝簡報和影片處理相關套件...")
!pip install python-pptx
!pip install pdf2image
!pip install moviepy
!sudo apt-get install poppler-utils libreoffice

print("安裝完成！")

In [ ]:
# @title 2. 上傳您的 PowerPoint 和聲音檔案
# @markdown 請執行此儲存格，然後點擊「選擇檔案」按鈕來上傳您的 .pptx 簡報檔案和 .wav 聲音樣本。
from google.colab import files
import os

print("請上傳您的 PowerPoint (.pptx) 檔案：")
uploaded_ppt = files.upload()
ppt_filename = list(uploaded_ppt.keys())[0]
os.rename(ppt_filename, 'presentation.pptx')
print(f"已上傳簡報檔案：{ppt_filename}")

print("\n請上傳您的聲音樣本 (.wav) 檔案 (建議5-15秒)：")
uploaded_audio = files.upload()
audio_filename = list(uploaded_audio.keys())[0]
os.rename(audio_filename, 'reference.wav')
print(f"已上傳聲音樣本：{audio_filename}")

In [ ]:
# @title 3. 處理 PowerPoint 檔案
# @markdown 這個儲存格會執行兩個主要任務：
# @markdown 1. 將您的 .pptx 檔案轉換為 PDF，然後再轉存為一系列的圖片 (每張投影片一張)。
# @markdown 2. 提取每張投影片下方的「演講者備忘稿」，這些文字將被用來生成語音。

import os
from pptx import Presentation
from pdf2image import convert_from_path
import subprocess

# 建立輸出資料夾
os.makedirs('slides', exist_ok=True)
os.makedirs('audio_outputs', exist_ok=True)
os.makedirs('video_outputs', exist_ok=True)

# 將 PPTX 轉換為 PDF
print("正在將 PowerPoint 轉換為 PDF...")
subprocess.run(['libreoffice', '--headless', '--convert-to', 'pdf', 'presentation.pptx'])

# 將 PDF 轉換為圖片
print("正在將 PDF 轉換為圖片...")
images = convert_from_path('presentation.pdf')
for i, image in enumerate(images):
    image.save(f'slides/slide_{i:03d}.png', 'PNG')
print(f"已成功將 {len(images)} 張投影片轉換為圖片。")

# 提取筆記
print("\n正在提取投影片筆記...")
prs = Presentation('presentation.pptx')
slide_notes = []
for i, slide in enumerate(prs.slides):
    notes = slide.notes_slide.notes_text_frame.text if slide.has_notes_slide else ""
    slide_notes.append(notes)
    print(f"投影片 {i+1} 的筆記： {notes}")

print("\nPowerPoint 處理完成！")

In [ ]:
# @title 4. 使用 F5-TTS 生成語音
# @markdown 現在，我們將使用您上傳的聲音樣本為每一張投影片的筆記生成語音檔。
# @markdown 這個過程可能需要一些時間，具體取決於您的簡報長度。

import torch
import subprocess
import shlex

for i, note in enumerate(slide_notes):
    if note.strip():
        print(f"正在為投影片 {i+1} 生成語音...")
        output_path = f"/content/F5-TTS/audio_outputs/slide_{i:03d}.wav"
        # Pass command as a string and use shell=True to handle quotes and special characters
        # Use shlex.quote to properly escape the text
        # Corrected argument from --output_path to --output_file
        command = f'f5-tts_infer-cli --model F5TTS_v1_Base --ref_audio "/content/F5-TTS/reference.wav" --gen_text {shlex.quote(note)} --output_file "{output_path}"'
        try:
            # Capture stderr to get more detailed error messages
            result = subprocess.run(command, check=True, shell=True, capture_output=True, text=True)
            print(f"投影片 {i+1} 的語音已儲存至 {output_path}")
        except subprocess.CalledProcessError as e:
            print(f"為投影片 {i+1} 生成語音時發生錯誤: {e}")
            print(f"錯誤輸出:\n{e.stderr}") # Print stderr for detailed error
            # You might want to add more error handling here, e.g., skip this slide or stop.
            continue # Skip to the next slide if an error occurs

    else:
        print(f"投影片 {i+1} 沒有筆記，跳過語音生成。")

torch.cuda.empty_cache()
print("\n所有語音檔案已生成完畢！")

In [ ]:
# @title 5. 合成影片片段
# @markdown 此儲存格會將每張投影片的圖片和其對應的語音檔結合成一個個獨立的影片片段。

from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips
import glob

video_clips = []
slide_images = sorted(glob.glob('slides/slide_*.png'))

for i, slide_image_path in enumerate(slide_images):
    audio_path = f'audio_outputs/slide_{i:03d}.wav'
    video_path = f'video_outputs/slide_{i:03d}.mp4'

    # 檢查對應的音訊是否存在
    if os.path.exists(audio_path):
        print(f"正在處理投影片 {i+1}...")
        with AudioFileClip(audio_path) as audio_clip:
            # 建立圖片影片，長度與音訊相同
            image_clip = ImageClip(slide_image_path, duration=audio_clip.duration)
            # 將音訊設定到圖片影片上
            video_clip = image_clip.set_audio(audio_clip)
            video_clip.fps = 24  # 設定影片的幀率
            video_clip.write_videofile(video_path, codec='libx264')
            video_clips.append(video_clip)
    else:
        # 如果沒有音訊，可以設定一個預設的顯示時間，例如5秒
        print(f"投影片 {i+1} 沒有對應的音訊，將以5秒的靜態圖片呈現。")
        image_clip = ImageClip(slide_image_path, duration=5)
        image_clip.fps = 24
        image_clip.write_videofile(video_path, codec='libx264')
        video_clips.append(image_clip)

print("\n所有獨立的影片片段已生成！")

In [ ]:
# @title 6. 合併影片並提供下載
# @markdown 最後，我們將所有投影片的影片片段合併成一個完整的簡報影片。
# @markdown 完成後，將會出現一個下載連結。

from moviepy.editor import VideoFileClip, concatenate_videoclips
import glob
import os
from google.colab import files

clip_paths = sorted(glob.glob('video_outputs/slide_*.mp4'))

# Load video clips from the generated files
video_clips = [VideoFileClip(clip_path) for clip_path in clip_paths]

if video_clips:
    final_clip = concatenate_videoclips(video_clips)

    final_video_path = 'final_presentation_video.mp4'
    final_clip.write_videofile(final_video_path, codec='libx264')

    print(f"\n🎉 恭喜！您的簡報影片已成功生成！")
    print(f"檔案儲存於：{final_video_path}")

    # Provide download link
    files.download(final_video_path)

    # Close the video clips to free up resources
    for clip in video_clips:
        clip.close()
else:
    print("沒有找到任何影片片段，無法合併。請檢查前面的步驟是否成功。")